# Real Data Based Simulator

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

import os
import time
import random
import joblib

import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostRegressor, CatBoostClassifier

Duplicate key in file PosixPath('/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 768 ('font.family  : sans-serif')
Duplicate key in file PosixPath('/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 769 ('font.sans-serif : SimHei, DejaVu Sans, Bitstream Vera Sans, Computer Modern Sans Serif, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif')
Duplicate key in file PosixPath('/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 770 ('axes.unicode_minus : False')


In [2]:
kde = joblib.load('model_x.pkl')
model_y = joblib.load('model_y.pkl')

In [3]:
def sample(n_samples=10000, coefs=[0,0]):
    # generate features
    Xs = kde.resample(n_samples).T.clip(min=0)
    # generate actions
    logits = Xs.dot(coefs)
    probs = 1.0/(1.0 + np.exp(-logits))
    Ts = np.random.binomial(n=1, p=probs)
    # generate responses with some noise
    XTs = np.hstack([Xs, Ts[:,np.newaxis]])
    Ys = model_y.predict(XTs) + np.random.randn(n_samples)*0.1
    return Xs, Ts, Ys

In [4]:
for ii in range(20):
    X1, T1, Y1 = sample(1000, [0,0])
    data1 = np.hstack([X1, T1[:,None], Y1[:,None]])
    
    X2, T2, Y2 = sample(1000, [1,-1])
    data2 = np.hstack([X2, T2[:,None], Y2[:,None]])
    
    df_b = pd.DataFrame(data1, columns=['X1','X2','T','Y'])
    df_e = pd.DataFrame(data2, columns=['X1','X2','T','Y'])
    
    df_b.to_csv('simulator_data/df_b_{}.csv'.format(ii), index=False)
    df_e.to_csv('simulator_data/df_e_{}.csv'.format(ii), index=False)